<a href="https://colab.research.google.com/github/manal-t07/Neural-Networks/blob/main/car_color_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install colorthief
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from colorthief import ColorThief
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, BatchNormalization, Lambda, AveragePooling2D, ZeroPadding2D, Reshape, Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

ModuleNotFoundError: ignored

In [ ]:
tf.test.is_built_with_cuda()

NameError: ignored

In [ ]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# Folder containing all the training data.
car_train = "Data/car_data/car_data/train/"
print('There are', len(os.listdir(car_train)),'folders in the training dataset')

In [ ]:
mylist = []
for labels in os.listdir(car_train):
    for label in os.listdir(car_train+labels): 
        car_add = car_train + labels + '/' + label
        car_value = (labels, car_add)                
        if car_value not in mylist:
            mylist.append(car_value)
            
train_df = pd.DataFrame(np.array(mylist), columns= ["car", "image path"])

In [ ]:
# Create train_df
train_df.sample(10)

In [ ]:
# Folder containng all the files for the test set. 
car_test= "Data/car_data/car_data/test/"
print('There are', len(os.listdir(car_test)),'folders in the test dataset')

In [ ]:
# Create test_df
mylist2 = []
for labels in os.listdir(car_test):
    for label in os.listdir(car_test+labels): 
        car_add = car_test + labels + '/' + label
        car_value = (labels, car_add)                
        if car_value not in mylist:
            mylist2.append(car_value)   

test_df = pd.DataFrame(np.array(mylist2), columns= ["car", "image path"])

In [ ]:
test_df.sample(10)

In [ ]:
# Load in colors.csv and create df with index as color
clr_headers = ['color', 'full_name', 'hex', 'r', 'g', 'b'] # list for header
colors = pd.read_csv('colors.csv', names=clr_headers, index_col = 'color') # Load in csv
colors['rgb'] = list(zip(colors['r'], colors['g'], colors['b'])) # combine r/g/b as rgb tuple
colors_df = colors.drop(columns=['full_name', 'hex', 'r', 'g', 'b']) # drop old columns and save to new df
colors_dict = dict(zip(colors_df.index, colors_df['rgb'])) # create dictionary with color(key):rgb(value)

def classify_color(filepath):
    # Color thief will find the most dominant rgb color in an image
    color_thief = ColorThief(filepath)
    dominant_color = color_thief.get_color(quality=1)

    # Manhattan distance between "classification points"
    manhattan = lambda x,y : abs(x[0] - y[0]) + abs(x[1] - y[1]) + abs(x[2] - y[2]) 
    
    # Find the best color in dictionary and find Manhattan distance
    distances = {k: manhattan(v, dominant_color) for k, v in colors_dict.items()}
    
    # Minimum distance to color point
    color = min(distances, key=distances.get) 
    return color

In [ ]:
# Will not re-run. Takes 2.5 hours.
# train_df['color'] = train_df['image path'].apply(classify_color)

In [ ]:
# # Save to csv
# train_df.to_csv('train_df_color')

In [ ]:
# Load in categorized data
train_df = pd.read_csv('train_df_color.csv')
train_df = train_df[['car', 'image path', 'color', 'car label']]

In [ ]:
# Create column for car label based on color
train_df['car_label'] = train_df.color.astype("category").cat.codes
train_df['car_label'] = train_df['car_label'].astype("str")

In [ ]:
# Most frequent colors
train_df.groupby(train_df['car_label']).count().sort_values(by='color', ascending=False).head(10)


In [ ]:
train_img_df = train_df[['image path', 'car_label']].copy()
train_img_df.sample(5)

In [ ]:
train_img_df.shape

In [ ]:
# # Will not re-run. Takes 2.5 hours.
# test_df['color'] = test_df['image path'].apply(classify_color)

In [ ]:
# # Save to csv
# test_df.to_csv('Data/test_df_color')

In [ ]:
test_df = pd.read_csv("test_df_color.csv")

In [ ]:
test_df['car_label'] = test_df.color.astype("category").cat.codes
test_df['car_label'] = test_df['car_label'].astype("str")

In [ ]:
test_df.head()

In [ ]:
test_df.groupby(test_df['car_label']).count().sort_values(by='color', ascending=False)

In [ ]:
test_img_df = test_df[['image path', 'car_label']].copy()
test_img_df.sample(5)

In [ ]:
test_img_df.shape